<a href="https://colab.research.google.com/github/Jonnybobba/NBA_Analysis/blob/Joels_Branch/NBAData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.co

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-02-25 05:51:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.75MB/s    in 0.5s    

2021-02-25 05:51:54 (1.75 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://nbadatabasebucket.s3.amazonaws.com/Team_00-20.csv"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("Team_00-20.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df.show(10)

+----+--------------------+---+-----+----+----+-----+---+----+-----+----+----+-----+----+----+-----+----+----+----+----+----+---+----+----+-----+--------+------------+
|Year|                Team|  G|   MP|  FG| FGA|  FG%| 3P| 3PA|  3P%|  2P| 2PA|  2P%|  FT| FTA|  FT%| ORB| DRB| TRB| AST| STL|BLK| TOV|  PF|  PTS|Playoffs|Championship|
+----+--------------------+---+-----+----+----+-----+---+----+-----+----+----+-----+----+----+-----+----+----+----+----+----+---+----+----+-----+--------+------------+
|2000|    Sacramento Kings| 82|20080|39.7|88.2|0.449|6.1|17.1|0.354|33.6|71.1|0.472|20.3|26.3|0.771|12.5|34.2|46.7|23.4|10.0|5.5|15.5|20.2|105.6|    true|       false|
|2000|     Milwaukee Bucks| 82|19780|41.0|89.5|0.458|7.4|19.5|0.379|33.6|70.0| 0.48|19.4|24.7|0.787|12.8|32.9|45.8|24.3| 8.8|5.1|14.8|25.4|108.8|    true|       false|
|2000|  Los Angeles Lakers| 82|19905|40.9|87.9|0.465|5.8|16.8|0.344|35.1|71.1|0.494|20.9|30.7|0.683|14.3|33.9|48.2|24.8| 7.4|6.4|15.6|24.6|108.4|    true|      

In [5]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://nbadatabasebucket.s3.amazonaws.com/Opp_00-20.csv"
spark.sparkContext.addFile(url)

df2 = spark.read.option('header', 'true').csv(SparkFiles.get("Opp_00-20.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
df2.show(10)

+----+--------------------+---+-----+----+----+-----+---+----+-----+----+----+-----+----+----+-----+----+----+----+----+---+---+----+----+-----+---------+------------+
|Year|                Team|  G|   MP|  FG| FGA|  FG%| 3P| 3PA|  3P%|  2P| 2PA|  2P%|  FT| FTA|  FT%| ORB| DRB| TRB| AST|STL|BLK| TOV|  PF|  PTS|Playoffs2|Championship|
+----+--------------------+---+-----+----+----+-----+---+----+-----+----+----+-----+----+----+-----+----+----+----+----+---+---+----+----+-----+---------+------------+
|2000|     New York Knicks| 82|19905|35.7|85.7|0.417|6.2|17.6|0.352|29.5|68.1|0.434|20.6|28.0|0.733|12.1|33.6|45.8|21.9|8.3|4.7|16.9|24.4| 98.2|     true|       false|
|2000|          Miami Heat| 82|19880|37.4|86.8|0.431|5.0|15.1|0.331|32.5|71.7|0.452|18.6|25.2|0.737|12.7|35.0|47.7|19.7|8.3|5.9|17.7|25.3| 98.5|     true|       false|
|2000|   San Antonio Spurs| 82|19830|38.3|91.5|0.419|4.6|14.1|0.329|33.7|77.4|0.435|16.7|22.5|0.741|13.1|32.9|45.9|21.8|8.1|5.8|15.0|26.3| 98.0|     true|      

In [6]:
import pandas as pd

In [7]:
Team_DF = df.toPandas()
Team_DF.head()

,Year,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Playoffs,Championship
0,2000,Sacramento Kings,82,20080,39.7,88.2,0.449,6.1,17.1,0.354,33.6,71.1,0.472,20.3,26.3,0.771,12.5,34.2,46.7,23.4,10.0,5.5,15.5,20.2,105.6,True,False
1,2000,Milwaukee Bucks,82,19780,41.0,89.5,0.458,7.4,19.5,0.379,33.6,70.0,0.480,19.4,24.7,0.787,12.8,32.9,45.8,24.3,8.8,5.1,14.8,25.4,108.8,True,False
2,2000,Los Angeles Lakers,82,19905,40.9,87.9,0.465,5.8,16.8,0.344,35.1,71.1,0.494,20.9,30.7,0.683,14.3,33.9,48.2,24.8,7.4,6.4,15.6,24.6,108.4,True,True
3,2000,Dallas Mavericks,82,19805,40.1,87.3,0.459,6.7,17.6,0.381,33.4,69.7,0.479,20.2,25.4,0.794,10.8,33.4,44.2,22.6,8.0,6.4,14.8,24.8,107.1,True,False
4,2000,Toronto Raptors,82,19955,40.3,92.2,0.437,5.7,15.4,0.369,34.6,76.8,0.451,19.6,26.2,0.747,14.8,33.4,48.2,26.5,7.9,6.9,14.3,23.1,105.9,True,False


In [8]:
Opp_DF = df2.toPandas()
Opp_DF.head()

,Year,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Playoffs2,Championship
0,2000,New York Knicks,82,19905,35.7,85.7,0.417,6.2,17.6,0.352,29.5,68.1,0.434,20.6,28.0,0.733,12.1,33.6,45.8,21.9,8.3,4.7,16.9,24.4,98.2,True,False
1,2000,Miami Heat,82,19880,37.4,86.8,0.431,5.0,15.1,0.331,32.5,71.7,0.452,18.6,25.2,0.737,12.7,35.0,47.7,19.7,8.3,5.9,17.7,25.3,98.5,True,False
2,2000,San Antonio Spurs,82,19830,38.3,91.5,0.419,4.6,14.1,0.329,33.7,77.4,0.435,16.7,22.5,0.741,13.1,32.9,45.9,21.8,8.1,5.8,15.0,26.3,98.0,True,False
3,2000,Charlotte Hornets,82,19880,37.4,87.8,0.426,5.3,14.6,0.360,32.1,73.2,0.439,19.5,25.8,0.756,12.0,32.9,44.9,23.6,8.3,5.1,16.3,23.9,99.5,True,False
4,2000,Philadelphia 76ers,82,19855,38.3,89.4,0.429,5.7,16.7,0.342,32.6,72.7,0.448,16.6,22.1,0.749,13.1,31.6,44.7,23.6,9.0,6.1,16.8,25.9,98.9,True,False


In [9]:
team_X = Team_DF.drop(columns=["Playoffs", "Championship"])
team_X

,Year,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2000,Sacramento Kings,82,20080,39.7,88.2,0.449,6.1,17.1,0.354,33.6,71.1,0.472,20.3,26.3,0.771,12.5,34.2,46.7,23.4,10.0,5.5,15.5,20.2,105.6
1,2000,Milwaukee Bucks,82,19780,41.0,89.5,0.458,7.4,19.5,0.379,33.6,70.0,0.480,19.4,24.7,0.787,12.8,32.9,45.8,24.3,8.8,5.1,14.8,25.4,108.8
2,2000,Los Angeles Lakers,82,19905,40.9,87.9,0.465,5.8,16.8,0.344,35.1,71.1,0.494,20.9,30.7,0.683,14.3,33.9,48.2,24.8,7.4,6.4,15.6,24.6,108.4
3,2000,Dallas Mavericks,82,19805,40.1,87.3,0.459,6.7,17.6,0.381,33.4,69.7,0.479,20.2,25.4,0.794,10.8,33.4,44.2,22.6,8.0,6.4,14.8,24.8,107.1
4,2000,Toronto Raptors,82,19955,40.3,92.2,0.437,5.7,15.4,0.369,34.6,76.8,0.451,19.6,26.2,0.747,14.8,33.4,48.2,26.5,7.9,6.9,14.3,23.1,105.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,2019,New York Knicks,66,15965,40.2,89.9,0.447,9.6,28.5,0.337,30.6,61.4,0.499,16.4,23.6,0.694,12.1,34.7,46.8,22.2,7.7,4.8,14.4,22.3,106.5
592,2019,Cleveland Cavaliers,65,15725,40.5,88.4,0.458,11.2,32.0,0.351,29.3,56.4,0.519,15.2,20.0,0.758,10.9,33.6,44.5,23.2,6.9,3.3,16.6,18.4,107.5
593,2019,Chicago Bulls,65,15675,39.5,88.5,0.447,12.2,35.1,0.348,27.3,53.5,0.511,15.5,20.5,0.755,10.4,31.4,41.8,23.2,10.0,4.1,15.4,21.8,106.7
594,2019,Golden State Warriors,65,15725,38.2,87.2,0.438,10.3,30.9,0.334,27.9,56.3,0.495,18.5,23.0,0.803,9.8,32.5,42.4,25.3,8.1,4.5,14.7,19.8,105.2


In [10]:
from sklearn.preprocessing import LabelEncoder
team_encoder = LabelEncoder()
team_encoder.fit(team_X["Team"])
team_X["Team"] = team_encoder.transform(team_X["Team"])
team_X

,Year,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2000,29,82,20080,39.7,88.2,0.449,6.1,17.1,0.354,33.6,71.1,0.472,20.3,26.3,0.771,12.5,34.2,46.7,23.4,10.0,5.5,15.5,20.2,105.6
1,2000,17,82,19780,41.0,89.5,0.458,7.4,19.5,0.379,33.6,70.0,0.480,19.4,24.7,0.787,12.8,32.9,45.8,24.3,8.8,5.1,14.8,25.4,108.8
2,2000,14,82,19905,40.9,87.9,0.465,5.8,16.8,0.344,35.1,71.1,0.494,20.9,30.7,0.683,14.3,33.9,48.2,24.8,7.4,6.4,15.6,24.6,108.4
3,2000,7,82,19805,40.1,87.3,0.459,6.7,17.6,0.381,33.4,69.7,0.479,20.2,25.4,0.794,10.8,33.4,44.2,22.6,8.0,6.4,14.8,24.8,107.1
4,2000,32,82,19955,40.3,92.2,0.437,5.7,15.4,0.369,34.6,76.8,0.451,19.6,26.2,0.747,14.8,33.4,48.2,26.5,7.9,6.9,14.3,23.1,105.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,2019,23,66,15965,40.2,89.9,0.447,9.6,28.5,0.337,30.6,61.4,0.499,16.4,23.6,0.694,12.1,34.7,46.8,22.2,7.7,4.8,14.4,22.3,106.5
592,2019,6,65,15725,40.5,88.4,0.458,11.2,32.0,0.351,29.3,56.4,0.519,15.2,20.0,0.758,10.9,33.6,44.5,23.2,6.9,3.3,16.6,18.4,107.5
593,2019,5,65,15675,39.5,88.5,0.447,12.2,35.1,0.348,27.3,53.5,0.511,15.5,20.5,0.755,10.4,31.4,41.8,23.2,10.0,4.1,15.4,21.8,106.7
594,2019,10,65,15725,38.2,87.2,0.438,10.3,30.9,0.334,27.9,56.3,0.495,18.5,23.0,0.803,9.8,32.5,42.4,25.3,8.1,4.5,14.7,19.8,105.2


In [11]:
team_X_train = team_X[team_X["Year"] < 2019 ]
team_X_train

,Year,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,2000,29,82,20080,39.7,88.2,0.449,6.1,17.1,0.354,33.6,71.1,0.472,20.3,26.3,0.771,12.5,34.2,46.7,23.4,10.0,5.5,15.5,20.2,105.6
1,2000,17,82,19780,41.0,89.5,0.458,7.4,19.5,0.379,33.6,70.0,0.480,19.4,24.7,0.787,12.8,32.9,45.8,24.3,8.8,5.1,14.8,25.4,108.8
2,2000,14,82,19905,40.9,87.9,0.465,5.8,16.8,0.344,35.1,71.1,0.494,20.9,30.7,0.683,14.3,33.9,48.2,24.8,7.4,6.4,15.6,24.6,108.4
3,2000,7,82,19805,40.1,87.3,0.459,6.7,17.6,0.381,33.4,69.7,0.479,20.2,25.4,0.794,10.8,33.4,44.2,22.6,8.0,6.4,14.8,24.8,107.1
4,2000,32,82,19955,40.3,92.2,0.437,5.7,15.4,0.369,34.6,76.8,0.451,19.6,26.2,0.747,14.8,33.4,48.2,26.5,7.9,6.9,14.3,23.1,105.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,2018,16,82,19730,40.3,89.4,0.450,11.5,32.9,0.349,28.8,56.5,0.509,15.3,22.1,0.695,11.4,35.6,47.1,24.7,7.8,5.5,15.0,21.2,107.3
562,2018,5,82,19905,39.8,87.8,0.453,9.1,25.9,0.351,30.7,61.9,0.496,16.2,20.6,0.783,8.7,34.1,42.8,21.9,7.3,4.3,14.1,20.3,104.8
563,2018,23,82,19780,38.2,88.2,0.433,10.0,29.5,0.340,28.2,58.7,0.479,18.1,23.8,0.759,10.4,34.2,44.7,20.1,6.8,5.1,14.0,20.9,104.5
564,2018,6,82,19755,40.1,90.3,0.444,10.6,30.0,0.355,29.4,60.3,0.488,16.9,21.3,0.792,11.0,32.9,44.0,21.3,6.7,2.5,13.9,20.6,107.7


In [12]:
team_X_test = team_X[team_X["Year"] == 2019 ]
team_results = team_X_test[["Year", "Team"]]
team_results["Team"] = team_encoder.inverse_transform(team_results["Team"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
from sklearn.preprocessing import LabelEncoder
result_encoder = LabelEncoder()
team_playoffs = Team_DF[["Year","Playoffs"]]
result_encoder.fit(team_playoffs["Playoffs"])
team_playoffs["Playoffs"] = result_encoder.transform(team_playoffs["Playoffs"])
team_playoffs

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Year,Playoffs
0,2000,1
1,2000,1
2,2000,1
3,2000,1
4,2000,1
...,...,...
591,2019,0
592,2019,0
593,2019,0
594,2019,0


In [14]:
team_playoffs_train = team_playoffs[team_playoffs["Year"] < 2019]
team_playoffs_test = team_playoffs[team_playoffs["Year"] == 2019]
team_playoffs_test

,Year,Playoffs
566,2019,1
567,2019,1
568,2019,1
569,2019,1
570,2019,1
571,2019,0
572,2019,0
573,2019,0
574,2019,0
575,2019,1


In [15]:
from sklearn.preprocessing import LabelEncoder

team_champ = Team_DF[["Year","Championship"]]
team_champ["Championship"] = result_encoder.transform(team_champ["Championship"])
team_champ

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Year,Championship
0,2000,0
1,2000,0
2,2000,1
3,2000,0
4,2000,0
...,...,...
591,2019,0
592,2019,0
593,2019,0
594,2019,0


In [16]:
team_champ_train = team_champ[team_champ["Year"] < 2019]
team_champ_test = team_champ[team_champ["Year"] == 2019]
team_champ_test

,Year,Championship
566,2019,0
567,2019,0
568,2019,0
569,2019,0
570,2019,0
571,2019,0
572,2019,0
573,2019,0
574,2019,0
575,2019,0


In [17]:
# Drop the Year column
team_X_test = team_X_test.drop(columns = ["Year"])
team_X_train = team_X_train.drop(columns = ["Year"])
team_playoffs_test = team_playoffs_test.drop(columns = ["Year"])
team_playoffs_train = team_playoffs_train.drop(columns = ["Year"])
team_champ_test = team_champ_test.drop(columns = ["Year"])
team_champ_train = team_champ_train.drop(columns = ["Year"])

In [18]:
# Reshape playoffs and champ dbs
team_playoffs_test_values = team_playoffs_test["Playoffs"].values.reshape(-1,1)
team_playoffs_train_values = team_playoffs_train["Playoffs"].values.reshape(-1,1)
team_champ_test_values = team_champ_test["Championship"].values.reshape(-1,1)
team_champ_train_values = team_champ_train["Championship"].values.reshape(-1,1)

In [19]:
print(team_playoffs_test_values.shape, team_playoffs_train_values.shape, team_champ_test_values.shape, team_champ_train_values.shape, team_X_test.shape, team_X_train.shape  )

(30, 1) (566, 1) (30, 1) (566, 1) (30, 24) (566, 24)


In [20]:
from sklearn.preprocessing import StandardScaler

team_X_scaler = StandardScaler().fit(team_X_train)
team_playoffs_scaler = StandardScaler().fit(team_playoffs_train_values)
team_champ_scaler = StandardScaler().fit(team_champ_train_values)

In [21]:
team_X_test_scaled = team_X_scaler.transform(team_X_test)
team_X_train_scaled = team_X_scaler.transform(team_X_train)
team_playoffs_test_scaled = team_playoffs_scaler.transform(team_playoffs_test_values)
team_playoffs_train_scaled = team_playoffs_scaler.transform(team_playoffs_train_values)
team_champ_test_scaled = team_champ_scaler.transform(team_champ_test_values)
team_champ_train_scaled = team_champ_scaler.transform(team_champ_train_values)

In [22]:
# Create a LinearRegression model and fit it to the scaled training data

### BEGIN SOLUTION
from sklearn.linear_model import LinearRegression
linear_playoff_model = LinearRegression()
linear_champ_model = LinearRegression()
linear_playoff_model.fit(team_X_train, team_playoffs_train_values)
linear_champ_model.fit(team_X_train, team_champ_train_values)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [23]:
champ_predictions = linear_champ_model.predict(team_X_test)
print(champ_predictions)

[[ 0.0651386 ]
 [ 0.16524553]
 [ 0.10763574]
 [-0.00020063]
 [ 0.07847434]
 [ 0.0774128 ]
 [ 0.08160077]
 [ 0.01919744]
 [ 0.0620463 ]
 [ 0.11475956]
 [ 0.11157978]
 [ 0.05446669]
 [ 0.15046747]
 [ 0.11093506]
 [ 0.13680533]
 [ 0.14520993]
 [-0.00375181]
 [ 0.08947158]
 [ 0.12037908]
 [ 0.08274418]
 [ 0.0049438 ]
 [ 0.01735994]
 [ 0.00117765]
 [-0.02766355]
 [-0.00069818]
 [-0.01206647]
 [-0.02449089]
 [ 0.04492425]
 [-0.0174405 ]
 [-0.02937002]]


In [24]:
from itertools import chain
champ_prediction = [x for x in chain.from_iterable(champ_predictions)]
print(champ_prediction)

[0.06513860143190264, 0.16524553197714908, 0.10763573965199491, -0.00020062596018277645, 0.078474340407519, 0.07741280314816201, 0.08160077481569594, 0.01919744190279893, 0.06204630191977856, 0.11475956374663276, 0.11157978144074931, 0.0544666935102196, 0.1504674654098732, 0.11093505933887648, 0.13680533212737167, 0.1452099311851356, -0.0037518061129571123, 0.08947158331808858, 0.12037907586218033, 0.08274417530248535, 0.0049438040383851245, 0.017359943781405818, 0.001177653752044705, -0.027663552246972145, -0.0006981813190094499, -0.012066466495122441, -0.024490889955949058, 0.044924248189735394, -0.017440496765727787, -0.029370017491906708]


In [25]:
champ_max = max(champ_prediction)
print(champ_max)

0.16524553197714908


In [26]:
champ_prediction_bool = []
for i in champ_prediction:
  if i == champ_max:
    champ_prediction_bool.append(True)
  else:
    champ_prediction_bool.append(False)
print(champ_prediction_bool)

[False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [27]:
team_results["Value"] = champ_prediction
team_results["Rank"] = team_results["Value"].rank(method = 'max', ascending = False)
team_results["Champion"] = champ_prediction_bool
team_results

,Year,Team,Value,Rank,Champion
566,2019,Dallas Mavericks,0.065139,15.0,False
567,2019,Milwaukee Bucks,0.165246,1.0,True
568,2019,Portland Trail Blazers,0.107636,9.0,False
569,2019,Houston Rockets,-0.000201,23.0,False
570,2019,Los Angeles Clippers,0.078474,13.0,False
571,2019,New Orleans Pelicans,0.077413,14.0,False
572,2019,Phoenix Suns,0.081601,12.0,False
573,2019,Washington Wizards,0.019197,19.0,False
574,2019,Memphis Grizzlies,0.062046,16.0,False
575,2019,Boston Celtics,0.114760,6.0,False


In [28]:
from google.colab import files
team_results.to_csv('linear_champ_results.csv')
files.download('linear_champ_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>